#  Naive Bayes: Clasificador de Mensajes Ham/Spam

In [1]:
!pip install scikit-learn

In [85]:
import re
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from collections import defaultdict
from collections import Counter

In [86]:
def limpiar_texto(texto):
    texto = re.sub(r'[^a-zA-Z\s]', '', texto)
    return texto.lower()

In [87]:
def entrenar_modelo(dataset):
    palabras_ham = defaultdict(int)
    palabras_spam = defaultdict(int)
    total_ham = 0
    total_spam = 0

    for mensaje, categoria in dataset:
        mensaje = limpiar_texto(mensaje)
        palabras = mensaje.split()

        if categoria == 'ham':
            total_ham += 1
            for palabra in palabras:
                palabras_ham[palabra] += 1
        elif categoria == 'spam':
            total_spam += 1
            for palabra in palabras:
                palabras_spam[palabra] += 1

    return palabras_ham, palabras_spam, total_ham, total_spam

def predecir_mensaje(mensaje, palabras_ham, palabras_spam, total_ham, total_spam, vocabulario):
    mensaje = limpiar_texto(mensaje)
    palabras = mensaje.split()

    prob_ham = 1.0
    prob_spam = 1.0

    for palabra in palabras:
        prob_ham *= (palabras_ham[palabra] + 1) / (total_ham + len(vocabulario))
        prob_spam *= (palabras_spam[palabra] + 1) / (total_spam + len(vocabulario))

    return 'ham' if prob_ham > prob_spam else 'spam'

vocabulario = set()
for mensaje, _ in train_data:
    mensaje = limpiar_texto(mensaje)
    vocabulario.update(mensaje.split())

vocabulario = list(vocabulario)

palabras_ham, palabras_spam, total_ham, total_spam = entrenar_modelo(train_data)

with open('entrenamiento.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

dataset = [(limpiar_texto(line.split('\t')[1]), line.split('\t')[0].strip()) for line in lines]
random.shuffle(dataset)  # Mezclar para aleatorizar

train_data, test_data = train_test_split(dataset, test_size=0.3, random_state=42)

y_true = [categoria for _, categoria in test_data]
y_pred = [predecir_mensaje(mensaje, palabras_ham, palabras_spam, total_ham, total_spam, vocabulario) for mensaje, _ in test_data]

In [88]:
accuracy = accuracy_score(y_true, y_pred)
report = classification_report(y_true, y_pred, zero_division=1)

### Métricas de desempeño

In [89]:
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)


Accuracy: 0.1437125748502994
Classification Report:
              precision    recall  f1-score   support

         ham       1.00      0.00      0.00      1430
        spam       0.14      1.00      0.25       240

    accuracy                           0.14      1670
   macro avg       0.57      0.50      0.13      1670
weighted avg       0.88      0.14      0.04      1670



In [70]:
total_mensajes_entrenamiento = len(train_data)
print("Total de mensajes en el conjunto de entrenamiento:", total_mensajes_entrenamiento)

total_mensajes_prueba = len(test_data)
print("Total de mensajes en el conjunto de prueba:", total_mensajes_prueba)

num_ham_entrenamiento = sum(1 for _, categoria in train_data if categoria == 'ham')
num_spam_entrenamiento = sum(1 for _, categoria in train_data if categoria == 'spam')
print("Número de mensajes 'ham' en el conjunto de entrenamiento:", num_ham_entrenamiento)
print("Número de mensajes 'spam' en el conjunto de entrenamiento:", num_spam_entrenamiento)

num_ham_prueba = sum(1 for _, categoria in test_data if categoria == 'ham')
num_spam_prueba = sum(1 for _, categoria in test_data if categoria == 'spam')
print("Número de mensajes 'ham' en el conjunto de prueba:", num_ham_prueba)
print("Número de mensajes 'spam' en el conjunto de prueba:", num_spam_prueba)



Total de mensajes en el conjunto de entrenamiento: 4452
Total de mensajes en el conjunto de prueba: 1113
Número de mensajes 'ham' en el conjunto de entrenamiento: 3854
Número de mensajes 'spam' en el conjunto de entrenamiento: 598
Número de mensajes 'ham' en el conjunto de prueba: 964
Número de mensajes 'spam' en el conjunto de prueba: 149


##  Lectura y limpieza del dataset

In [78]:
import re
import random
from sklearn.model_selection import train_test_split
from collections import defaultdict
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [79]:
def limpiar_texto(texto):
    texto = re.sub(r'[^a-zA-Z\s]', '', texto)
    return texto.lower()

In [80]:
def entrenar_modelo(dataset):
    palabras_ham = defaultdict(int)
    palabras_spam = defaultdict(int)
    total_ham = 0
    total_spam = 0

    for categoria, mensaje in dataset:
        mensaje = limpiar_texto(mensaje)
        palabras = mensaje.split()

        if categoria == 'ham':
            total_ham += 1
            for palabra in palabras:
                palabras_ham[palabra] += 1
        elif categoria == 'spam':
            total_spam += 1
            for palabra in palabras:
                palabras_spam[palabra] += 1

    return palabras_ham, palabras_spam, total_ham, total_spam

def predecir_mensaje(mensaje, palabras_ham, palabras_spam, total_ham, total_spam, vocabulario):
    mensaje = limpiar_texto(mensaje)
    palabras = mensaje.split()

    prob_ham = 1.0
    prob_spam = 1.0

    for palabra in palabras:
        prob_ham *= (palabras_ham[palabra] + 1) / (total_ham + len(vocabulario))
        prob_spam *= (palabras_spam[palabra] + 1) / (total_spam + len(vocabulario))

    return 'ham' if prob_ham > prob_spam else 'spam'

In [81]:
with open('entrenamiento.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

dataset = [(line.split('\t')[0].strip(), line.split('\t')[1].strip()) for line in lines]
random.shuffle(dataset)  

train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

In [82]:
vocabulario = set()
for _, mensaje in train_data:
    mensaje = limpiar_texto(mensaje)
    vocabulario.update(mensaje.split())

vocabulario = list(vocabulario)

palabras_ham, palabras_spam, total_ham, total_spam = entrenar_modelo(train_data)

y_true = [categoria for categoria, _ in test_data]
y_pred = [predecir_mensaje(mensaje, palabras_ham, palabras_spam, total_ham, total_spam, vocabulario) for _, mensaje in test_data]

In [83]:
accuracy = accuracy_score(y_true, y_pred)
conf_matrix = confusion_matrix(y_true, y_pred)
report = classification_report(y_true, y_pred, zero_division=1)

### Métricas de desempeño

In [84]:
print(f'Accuracy: {accuracy}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(report)

Accuracy: 0.9568733153638814
Confusion Matrix:
[[958   7]
 [ 41 107]]
Classification Report:
              precision    recall  f1-score   support

         ham       0.96      0.99      0.98       965
        spam       0.94      0.72      0.82       148

    accuracy                           0.96      1113
   macro avg       0.95      0.86      0.90      1113
weighted avg       0.96      0.96      0.95      1113



El modelo tiene una precisión del 95.69%, lo cual es alto. Sin embargo, es importante tener en cuenta que, en casos desbalanceados (como en la clasificación de spam), la Accuracy por sí sola puede no ser suficiente para evaluar completamente el rendimiento. 

Para la clase 'ham', se observa una alta precisión (96%) y recall (99%), lo que indica que el modelo clasifica eficazmente la mayoría de los mensajes 'ham'.

Para la clase 'spam', la precisión es del 94%, lo que significa que de los mensajes clasificados como 'spam', el 94% realmente son spam. Sin embargo, el recall es del 72%, lo que indica que el modelo no identifica todos los mensajes 'spam' existentes.